In [1]:
import matplotlib as plt
import numpy as np
import datetime
import calendar
import requests
import pandas as pd

In [3]:
cd Meerkat_data_new/

[Errno 2] No such file or directory: 'Meerkat_data_new/'
/data/Wind_data_project/Meerkat_data_new


# Retrieve sensor data from CAM via http request
- mgouws@sarao.ac.za, 07-08-2023


In [4]:
#Definitions




# Defined list of sensor portal URLs for reference:
SENSOR_PORTALS = ["http://portal.mkat.karoo.kat.ac.za/katstore/api/query",
                  "http://portal.mkat-rts.karoo.kat.ac.za/katstore/api/query",
                  "http://kat-flap-cpt.mkat.control.kat.ac.za/katstore/api/query",
                  "http://kat-flap-cpt.mkat-rts.control.kat.ac.za/katstore/api/query"]


def get_sensor(sensorname, start_time, end_time, timeout=100,
               store_url="http://portal.mkat.karoo.kat.ac.za/katstore/api/query"):
    """ Retrieves data from the CAM sensor database with an http request.
    
    Parameters
    ----------
    sensorname : string
        Name of the sensor to retrieve, e.g. 'm004_dig_l_band_rfcu_hpol_rf_power_in'
    start_time : unix timestamp
        Start time of sensor database search.
    end_time : unix timestamp
        End time of sensor database search.
    timeout : integer, optional
        Timeout time for http request, in seconds. (Default=100)
    store_url : string, optional
        URL of the desired sensor portal. (Default="http://portal.mkat.karoo.kat.ac.za/katstore/api/query")
        
    Returns
    -------
    times, values : array
        Retrieved timestamps and values of requested sensor.
        
    """
    params={'sensor': sensorname,  # name of the sensor.
            'start_time': start_time, # start time in sec
            'end_time': end_time, # end time in sec
            'limit':10000000}
    try:
        res = requests.get(store_url, params, timeout=timeout) # Submit the query
        res.raise_for_status()  # Raise exception on status errors
        # Data comes back as JSON with the correct mime type set so can be converted by Requests
        data_list = np.asarray([(r['sample_time'], r['value']) for r in res.json()['data']])
    except Exception as e:
        print("Failed to connect to %s: \n%s" % (store_url,e))
        data_list = np.array([])
    if (np.shape(data_list)[0] == 0):
        return (np.asarray([]), np.asarray([]))
    else:
        _t, _v = np.asarray(data_list[:,0], dtype=float), data_list[:,1] 
        return (_t, _v)



Divide data queries into one-month chunks in order not to overwhelm the server. Otherwise internal server errors get raised.



In [7]:
import matplotlib as plt
import numpy as np
import datetime
import calendar
import requests
import pandas as pd

In [11]:
sensor_list = [ "anc_air_pressure"]
years = range(2019,2024)
#years=[2023]
for sensor_name in sensor_list:
    print(sensor_name)
    for year in years:
        print(year)
        df = pd.DataFrame()
        for month in range(1,13):
            start_time = datetime.datetime(year, month, 1, 0, 0, 0, tzinfo=datetime.timezone.utc)
            last_DoM = calendar.monthrange(year, month)[1]
            end_time = datetime.datetime(year, month, last_DoM, 23, 59, 59, tzinfo=datetime.timezone.utc)
            print(start_time)
            timestamps, values = get_sensor(sensor_name, start_time.timestamp(), end_time.timestamp())
            df = pd.concat([df, pd.DataFrame({'Timestamp': timestamps, sensor_name: values})]) 
            df.info()
        print('exit loop')
        if df.empty:
            print('no values of %s for year %d'%(sensor_name,year))
        else:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'],unit='s')
            df.to_pickle('/data/Wind_data_project/Meerkat_data/'+sensor_name+'_'+str(year)+'.pkl')



anc_air_pressure
2019
2019-01-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267240 entries, 0 to 267239
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Timestamp         267240 non-null  float64
 1   anc_air_pressure  267240 non-null  float64
dtypes: float64(2)
memory usage: 4.1 MB
2019-02-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 505547 entries, 0 to 238306
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Timestamp         505547 non-null  float64
 1   anc_air_pressure  505547 non-null  float64
dtypes: float64(2)
memory usage: 11.6 MB
2019-03-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 773240 entries, 0 to 267692
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Time

<class 'pandas.core.frame.DataFrame'>
Int64Index: 843797 entries, 0 to 82377
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Timestamp         843797 non-null  float64
 1   anc_air_pressure  843797 non-null  float64
dtypes: float64(2)
memory usage: 19.3 MB
exit loop
2021
2021-01-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82558 entries, 0 to 82557
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Timestamp         82558 non-null  float64
 1   anc_air_pressure  82558 non-null  float64
dtypes: float64(2)
memory usage: 1.3 MB
2021-02-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 147086 entries, 0 to 64527
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Timestamp         147086 non-null  float64
 1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 772674 entries, 0 to 55740
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Timestamp         772674 non-null  float64
 1   anc_air_pressure  772674 non-null  float64
dtypes: float64(2)
memory usage: 17.7 MB
2022-12-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 827929 entries, 0 to 55254
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Timestamp         827929 non-null  float64
 1   anc_air_pressure  827929 non-null  float64
dtypes: float64(2)
memory usage: 18.9 MB
exit loop
2023
2023-01-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59408 entries, 0 to 59407
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Timestamp         59408 non-null  float64


In [ ]:
sensor_list = ["anc_mean_wind_speed", "anc_gust_wind_speed", "anc_wind_direction", "m000_windstow_active",
               "m058_windstow_active", "m059_windstow_active", "anc_air_temperature", "anc_air_relative_humidity", "anc_weather_rain"]
#years = range(2023,2024)
years=[2023]
for sensor_name in sensor_list:
    print(sensor_name)
    for year in years:
        print(year)
        df = pd.DataFrame()
        for month in range(1,13):
            start_time = datetime.datetime(year, month, 1, 0, 0, 0, tzinfo=datetime.timezone.utc)
            last_DoM = calendar.monthrange(year, month)[1]
            end_time = datetime.datetime(year, month, last_DoM, 23, 59, 59, tzinfo=datetime.timezone.utc)
            print(start_time)
            timestamps, values = get_sensor(sensor_name, start_time.timestamp(), end_time.timestamp())
            df = pd.concat([df, pd.DataFrame({'Timestamp': timestamps, sensor_name: values})]) 
            df.info()
        print('exit loop')
        if df.empty:
            print('no values of %s for year %d'%(sensor_name,year))
        else:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'],unit='s')
            df.to_pickle('/data/Wind_data_project/Meerkat_data_new/'+sensor_name+'_'+str(year)+'.pkl')


anc_mean_wind_speed
2023
2023-01-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2678396 entries, 0 to 2678395
Data columns (total 2 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Timestamp            float64
 1   anc_mean_wind_speed  float64
dtypes: float64(2)
memory usage: 40.9 MB
2023-02-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5069805 entries, 0 to 2391408
Data columns (total 2 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Timestamp            float64
 1   anc_mean_wind_speed  float64
dtypes: float64(2)
memory usage: 116.0 MB
2023-03-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7722609 entries, 0 to 2652803
Data columns (total 2 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Timestamp            float64
 1   anc_mean_wind_speed  float64
dtypes: float64(2)
memory usage: 176.8 MB
2023-04-01 00:00:00+00:00
<c

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731653 entries, 0 to 249953
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Timestamp           731653 non-null  float64
 1   anc_wind_direction  731653 non-null  float64
dtypes: float64(2)
memory usage: 16.7 MB
2023-04-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 969875 entries, 0 to 238221
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Timestamp           969875 non-null  float64
 1   anc_wind_direction  969875 non-null  float64
dtypes: float64(2)
memory usage: 22.2 MB
2023-05-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1212976 entries, 0 to 243100
Data columns (total 2 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Timestamp           1212976 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 4017
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Timestamp             8515 non-null   float64
 1   m058_windstow_active  8515 non-null   float64
dtypes: float64(2)
memory usage: 199.6 KB
2023-03-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12956 entries, 0 to 4440
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Timestamp             12956 non-null  float64
 1   m058_windstow_active  12956 non-null  float64
dtypes: float64(2)
memory usage: 303.7 KB
2023-04-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17274 entries, 0 to 4317
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Timestamp             17274 n

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250044 entries, 0 to 250043
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Timestamp            250044 non-null  float64
 1   anc_air_temperature  250044 non-null  float64
dtypes: float64(2)
memory usage: 3.8 MB
2023-02-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 472937 entries, 0 to 222892
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Timestamp            472937 non-null  float64
 1   anc_air_temperature  472937 non-null  float64
dtypes: float64(2)
memory usage: 10.8 MB
2023-03-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 719430 entries, 0 to 246492
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Timestamp            7

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2321147 entries, 0 to 102005
Data columns (total 2 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Timestamp                  float64
 1   anc_air_relative_humidity  float64
dtypes: float64(2)
memory usage: 53.1 MB
exit loop
anc_weather_rain
2023
2023-01-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2678387 entries, 0 to 2678386
Data columns (total 2 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Timestamp         float64
 1   anc_weather_rain  float64
dtypes: float64(2)
memory usage: 40.9 MB
2023-02-01 00:00:00+00:00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5069797 entries, 0 to 2391409
Data columns (total 2 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Timestamp         float64
 1   anc_weather_rain  float64
dtypes: float64(2)
memory usage: 116.0 MB
2023-03-01 00:00:00+00:00
<class 'pandas.core.fra